In [ ]:
!pip install torch
!pip install torchvision
!pip install matplotlib

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
test_data

In [ ]:
bs = 64
train_dl = DataLoader(training_data, batch_size=bs)
test_dl = DataLoader(test_data, batch_size=bs)


In [ ]:
train_dl.dataset[0][0].shape # 28 x 28 images

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_dl.dataset[0][0].numpy().squeeze(), cmap="gray")

In [ ]:
for X, y in test_dl:
  print(f'Shape of X [N, C, H, W]: {X.shape}')
  print(f'Shape of y [N]: {y.shape} {y.dtype}')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
class NN(nn.Module):
  def __init__(self):
    super(NN, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10),
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NN().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 0.001

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) # independent vars (img), dependent vars (actuals)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Bcakpropagate
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    # Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}/{epochs}\n")
  train(train_dl, model, loss_fn, optimizer)
  test(test_dl, model, loss_fn)

print("Done")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("saved model")

In [ ]:
model = NN()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
plt.imshow(train_dl.dataset[1][0].numpy().squeeze(), cmap="gray")

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    pred = model(x)
    print(pred)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
pred[0].argmax(0)